In [9]:
import geopandas as gpd
import shapely.geometry
import math
import matplotlib

In [15]:
def create_tiles(min_lat, max_lat, min_lon, max_lon, tile_size=1, padding=0.05):
    """
    Create a GeoDataFrame of 1x1 degree tiles over the area of interest,
    with a 0.05 degree overlap on the top and bottom edges.
    
    Parameters:
        min_lat (float): Minimum latitude of the area of interest.
        max_lat (float): Maximum latitude of the area of interest.
        min_lon (float): Minimum longitude of the area of interest.
        max_lon (float): Maximum longitude of the area of interest.
    
    Returns:
        gpd.GeoDataFrame: GeoDataFrame containing the tile polygons.
    """
    tiles = []
    for i in range(math.ceil((max_lon - min_lon)/(tile_size - 2*padding))):
        for j in range(math.ceil((max_lat - min_lat)/(tile_size - 2*padding))):
            ymin = min_lat + j*(tile_size-2*padding)
            ymax = ymin + tile_size
            xmin = min_lon + i*(tile_size-2*padding)
            xmax = xmin + tile_size
            tile = shapely.geometry.Polygon([
                (xmin, ymin),
                (xmax, ymin),
                (xmax, ymax),
                (xmin, ymax),
                (xmin, ymin)
            ])
            tiles.append(tile)
    
    return gpd.GeoDataFrame(geometry=tiles, crs="EPSG:4326")

In [ ]:
def check_tiles(tiles_gdf, land_path):
    """
    Filters tiles to retain only those that overlap with land areas.

    Parameters:
        tiles_gdf (gpd.GeoDataFrame): A GeoDataFrame containing the tile polygons.
        land_path (str): File path to the Natural Earth land polygons dataset (https://www.naturalearthdata.com/downloads/50m-physical-vectors/50m-land/)

    Returns:
        gpd.GeoDataFrame: A GeoDataFrame containing only the tiles that intersect land.
    """
    land_gdf = gpd.read_file(land_path)
    land_merged = land_gdf.dissolve().geometry.unary_union
    filtered_tiles = tiles_gdf[tiles_gdf.intersects(land_merged)]
    if filtered_tiles.empty:
        raise ValueError("No tiles overlap with the land polygons.")
    
    return filtered_tiles


In [3]:
min_lat = 23.5
max_lat = 35
min_lon = -110
max_lon = -103.2

In [33]:
filtered_tiles.empty

False

In [17]:
gdf = create_tiles(min_lat, max_lat, min_lon, max_lon)

In [43]:
for i, tile in filtered_tiles.iterrows():
    minx, miny, maxx, maxy = tile.geometry.bounds

In [ ]:
def main():
    parser = get_parser()
    args = parser.parse_args()

    # create tiles geodataframe
    tiles_gdf = create_tiles(args.aoi.minlat, args.aoi.maxlat, args.aoi.minlon, args.aoi.maxlon)
    land_path = '../data/polygons/ne_50m_land.shp'
    tiles_gdf = check_tiles(tiled_gdf, land_path)

    # set up matrix job
    tiles = []
    for i, tile in tiles_gdf.iterrows():
        minlon, minlat, maxlon, maxlat = tile.geometry.bounds
        shortname = f'{args.target_date}_{minlon:.{2}f}_{minlat:.{2}f}_{maxlon:.{2}f}_{maxlat:.{2}f}'
        tiles.append({'tile_date':args.target_date, 'minlon':minlon, 'minlat':minlat, 'maxlon':maxlon, 'maxlat':matlat, 'name':shortname})
    matrixJSON = f'{{"include":{json.dumps(tiles)}}}'
    print(f'number of tiles: {len(tiles)}')
    with open(os.environ['GITHUB_OUTPUT'], 'a') as f:
        print(f'MATRIX_PARAMS_COMBINATIONS={matrixJSON}', file=f)

In [ ]:
product_names = first_image_indices['s2:product_uri'].values.tolist()
print('\n'.join(product_names))

# Create Matrix Job Mapping (JSON Array)
pairs = []
for r in range(len(product_names) - int(args.npairs)):
    for s in range(1, int(args.npairs) + 1 ):
        img1_product_name = product_names[r]
        img2_product_name = product_names[r+s]
        shortname = f'{img1_product_name[11:19]}_{img2_product_name[11:19]}'
        pairs.append({'img1_product_name': img1_product_name, 'img2_product_name': img2_product_name, 'name':shortname})
matrixJSON = f'{{"include":{json.dumps(pairs)}}}'
print(f'number of image pairs: {len(pairs)}')

with open(os.environ['GITHUB_OUTPUT'], 'a') as f:
    print(f'IMAGE_DATES={product_names}', file=f)
    print(f'MATRIX_PARAMS_COMBINATIONS={matrixJSON}', file=f)